In [1]:
import Data
import regression
import dclassifier
import numpy as np
import pandas as pd


In [2]:

training_months = [pd.Period('2019-04', 'M'), pd.Period('2019-05', 'M'), pd.Period('2019-06', 'M'), pd.Period('2019-07', 'M')]

data = Data.Data()

prices = data.prices
prices = prices[prices.index.to_period('M').isin(training_months)]


In [4]:
thresholds = np.arange(0.0001, 0.0020, 0.0001)
currencies = prices.columns

# remove AUDUSD, because it frequently creates a regression expression that doesn't have DCl
currencies = [curr for curr in currencies if curr != 'AUDUSD']

combinations = [(threshold, currency) for threshold in thresholds for currency in currencies]

losses = {}
best_individual = {}
for combination in combinations:
    threshold, currency = combination

    events, dcclass = dclassifier.classify_split_timeseries(prices[currency], threshold)
    DCs_with_OSs = [(a['end'] - a['start'], b['start'] - a['end']) for (_, a), (_, b) in zip(events.iloc[:-1].iterrows(), events.iloc[1:].iterrows()) if b['prev_os']]

    loss, individual = regression.SRGP(DCs_with_OSs)

    best_individual[combination] = individual
    losses[combination] = loss

In [21]:
def print_node(node):
    string = ''
    if node.val:
        string += (node.val if isinstance(node.val, str) else str(round(node.val, 3)))
    if len(node.children):
        string += f" ({' , '.join([print_node(child) for child in node.children])})"
    return string


In [22]:
combination_best_loss = min(losses, key=losses.get)
best = best_individual[combination_best_loss]
print_node(best)

'+ (sin (sin (-6.147)) , log (- (DCl , -9.602)))'

In [23]:
example_dcl = 10
regression.eval_tree(best, example_dcl)

3.11111968728486

In [33]:
for combination in combinations:
    threshold, currency = combination
    print(f'currency: {currency}, threshold: {threshold:.4f}, loss: {losses[combination]:6.3f}, expression: {print_node(best_individual[combination])}')

currency: EURCHF, threshold: 0.0001, loss:  2.641, expression: log (+ (* (-7.806 , -2.28) , * (DCl , DCl)))
currency: EURJPY, threshold: 0.0001, loss:  1.993, expression: + (sin (sin (-6.147)) , log (- (DCl , -9.602)))
currency: EURUSD, threshold: 0.0001, loss:  2.442, expression: log (- (+ (DCl , 4.528) , + (-4.837 , -6.922)))
currency: USDCAD, threshold: 0.0001, loss:  2.384, expression: + (log (6.999) , cos (/ (-0.634 , DCl)))
currency: USDCHF, threshold: 0.0001, loss:  2.321, expression: log (- (* (-0.855 , -7.998) , * (-7.998 , DCl)))
currency: USDJPY, threshold: 0.0001, loss:  2.140, expression: log (+ (* (DCl , DCl) , - (6.308 , -4.803)))
currency: EURCHF, threshold: 0.0002, loss:  4.128, expression: 4.238
currency: EURJPY, threshold: 0.0002, loss:  2.875, expression: - (log (* (DCl , 7.589)) , - (+ (-5.44 , 3.852) , / (DCl , -2.495)))
currency: EURUSD, threshold: 0.0002, loss:  3.794, expression: log (+ (* (6.746 , 6.746) , * (DCl , DCl)))
currency: USDCAD, threshold: 0.0002, l